# Warsztat

Celem warsztatu jest napisanie skryptu, który przeanalizuje piosenki podanego wykonawcy pod kątem pogody w miejscach, o których wspomina ich tekst. Mając jedynie nazwę zespołu, musimy zdobyć teksty wszystkich jego piosenek, następnie poszukać w nich nazw miejsc, a na końcu sprawdzić jaka panuje tam obecnie pogoda.

## Potrzebne klucze

Warto zorganizować sobie miejsce na wszystkie klucze, jakie będą potrzebne do współpracy z API. Początek pliku wydaje się na nie odpowiednim miejscem:

In [ ]:
THEAUDIODB_KEY = '2'
GEOCODEXYZ_KEY = ''  # UZUPEŁNIJ!
OPENWEATHERMAPORG_KEY = ''  # UZUPEŁNIJ!

### Uwaga
Ponieważ serwis `AudioDB` doświadczał dość dużego obciążenia związanego z używaniem _publicznego_ klucza, wprowadzone zostały następujące ograniczenia:
- Publiczne API Key zostało zmienione z 1 na 2,
- Zapytanie możemy wykonać tylko **raz na dwie sekundy**

Więcej o tym problemie można zleźć w dokumentacji API: [klik](https://www.theaudiodb.com/api_guide.php)


## Pobranie listy piosenek

Listę piosenek możemy pobrać z [https://www.theaudiodb.com/](https://www.theaudiodb.com/), pod warunkiem że znamy ID płyt, na jakich te się znajdują. ID płyt poznamy natomiast dopiero, gdy będziemy znali ID artysty.

Trzeba zatem zacząć od wyszukania artysty po nazwie. Otwórz link i obejrzyj, jak wygląda odpowiedź: [https://www.theaudiodb.com/api/v1/json/2/search.php?s=coldplay](https://www.theaudiodb.com/api/v1/json/1/search.php?s=coldplay)

Jest to endpoint przeznaczony do wyszukiwania, więc należy spodziewać się **listy** pasujących zespołów - nawet gdy nazwa została podana idealnie, i został znaleziony tylko jeden.

Napiszmy funkcję, która przyjmuje nazwę zespołu, i zwraca słownik z kluczami `id` oraz `name` - lub `None`, gdy zespołu nie udało się znaleźć.

**Warto używać funkcji `print`, aby zorientować się, co skrypt robi w danej chwili.**

In [ ]:
import requests
from time import sleep

In [ ]:
def get_band(name):
    print('Pobieram dane o zespole...')
    band_details_response = requests.get(f'https://www.theaudiodb.com/api/v1/json/{THEAUDIODB_KEY}/search.php', {'s': name})
    band_details = band_details_response.json()

    sleep(2)  # wymagane przez dokumentację API: https://www.theaudiodb.com/api_guide.php

    if band_details['artists']:
        return {'id': band_details['artists'][0]['idArtist'], 'name': band_details['artists'][0]['strArtist']}
    else:
        return None

Powyższa funkcja zapewni nam ID artysty, którego użyjemy do pobrania listy jego albumów.

Mając ID artysty możemy założyć, że pobranie listy albumów się uda - pobieramy ją przecież z tego samego API. Napiszmy funkcję, która przyjmie ID artysty, a zwróci listę ID albumów - tylko one są nam potrzebne, aby finalnie pobrać nazwy piosenek artysty.

In [ ]:
def get_album_ids(band_id):
    print('Pobieram albumy...')
    albums_response = requests.get(f'https://theaudiodb.com/api/v1/json/{THEAUDIODB_KEY}/album.php', {'i': band_id})
    albums = albums_response.json()

    sleep(2)  # wymagane przez dokumentację API: https://www.theaudiodb.com/api_guide.php

    return [album['idAlbum'] for album in albums['album']]

Powyższa funkcja zwróci listę kilku-kilkudziesięciu identyfikatorów albumów. Napiszmy teraz funkcję, która potrafi pobrać piosenki z **jednego** albumu - później użyjemy jej w pętli, aby odpytać API o wszystkie albumy po kolei.

Tym razem interesują nas tylko nazwy piosenek - tego będzie od nas oczekiwać kolejne API.

In [ ]:
def get_tracks(album_id):
    print(f'Pobieram ścieżki z albumu {album_id}...')
    tracks_response = requests.get(f'https://theaudiodb.com/api/v1/json/{THEAUDIODB_KEY}/track.php', {'m': album_id})
    tracks = tracks_response.json()

    sleep(2)  # wymagane przez dokumentację API: https://www.theaudiodb.com/api_guide.php

    return [track['strTrack'] for track in tracks['track']]

## Pobranie tekstów piosenek

Po kilku próbach użycia API [https://api.lyrics.ovh/](https://api.lyrics.ovh/) zapewne zauważysz, że nie jest ono idealne. Wszystko działa dobrze, gdy tekst piosenki znajduje się w jego bazie danych. Problem zaczyna się, gdy tekstu tam nie ma - w takiej sytuacji API bardzo długo nie zamyka połączenia, i trzyma swojego klienta w niepewności...

Na szczęście biblioteka `requests` pozwala określić maksymalny czas, jaki jesteśmy skłonni poczekać na odpowiedź z serwera. [Z dokumentacji](https://requests.readthedocs.io/en/master/user/quickstart/#timeouts) dowiemy się, że funkcje biblioteki przyjmują argument `timeout` (w sekundach) - gdy taki czas minie, funkcja rzuci wyjątek zamiast zwracać odpowiedź.

Drugi problem to to, że nazwy zespołów i piosenek są przekazywane do API w adresie, ale nie w querystringu. Co jeśli zespół ma w swojej nazwie `?`? API na pewno źle zinterpretuje takie zapytanie. Rozwiązaniem jest zastąpienie wszystkich "niebezpiecznych" znaków ich kodami w formie `%xx` - tym zajmie się funkcja `quote` z modułu `urllib.parse`.

Gotowa funkcja zwraca tekst piosenki, lub `None` gdy nie udało się go znaleźć.

In [ ]:
from urllib.parse import quote

def get_lyrics(band, title):
    try:
        response = requests.get(f'https://api.lyrics.ovh/v1/{quote(band)}/{quote(title)}', timeout=15)
        sleep(2)  # wymagane przez dokumentację API: https://www.theaudiodb.com/api_guide.php

        return response.json()['lyrics']
    except Exception:
        print(f'Brak tekstu dla {band} - {title}')
        return None

## Pobranie miejsc z tekstu

Do pobrania nazw miast, wiosek, stanów i państw posłuży nam kolejne API: [https://geocode.xyz](https://geocode.xyz).

Wypróbuj to API, podając tekst, w którym są:
- dwa lub więcej miejsc (`"Warszawa i Kraków"`)
- jedno miejsce (`"Warszawa"`)
- brak miejsc (`"Nic"`)

Rozsądnie byłoby, gdyby zawsze była zwracana lista znalezionych miejsc - nawet jeśli miałaby być pusta.

Tak niestety nie jest! 
- Jeśli żadne miejsce nie zostanie znalezione, klucz `"match"` w ogóle nie pojawi się w odpowiedzi.
- Jeśli zostało znalezione tylko jedno miejsce, klucz `"match"` nie będzie zawierał listy - zamiast tego znajdzie się w nim od razu słownik z jedynym znalezionym miejscem.
- Jeśli zostało znalezionych więcej miejsc, klucz `"match"` będzie zawierał listę.

Zatem API zwraca wyniki na 3 różne sposoby - aby nasza funkcja była bardziej spójna, rozpoznamy każdą z tych trzech sytuacji i obsłużymy odpowiednio.

In [ ]:
def get_places(text):
    response = requests.post('https://geocode.xyz', {'scantext': text, 'geoit': 'json', 'sentiment': 'analysis', 'auth': GEOCODEXYZ_KEY})
    if response.status_code == 200:
        content = response.json()
        if 'match' not in content:
            print('Brak lokalizacji w tekście')
            return []
        if type(content['match']) == list:
            return [match['location'] for match in content['match']]
        else:
            return [content['match']['location']]
    else:
        print('Zapytanie do geocode.xyz wróciło ze statusem', response.status_code)
        return []

## Pobranie pogody

To API jest Ci już znane - zwraca odpowiedź ze statusem `200 OK` i pogodą dla podanego miejsca, lub odpowiedź `404 NOT FOUND` jeśli miejsce nie zostało znalezione.

Za pomocą `response.ok` sprawdzimy, z którą z tych sytuacji mamy do czynienia, i odpowiednio zwrócimy wynik.

In [ ]:
def get_weather(location):
    response = requests.get(f'http://api.openweathermap.org/data/2.5/weather', {'q': location, 'appid': OPENWEATHERMAPORG_KEY})
    if response.ok:
        return response.json()['weather'][0]['description']
    else:
        print('Nie znaleziono pogody dla:', location)
        return None

## Złożenie wszystkiego w całość

Nadszedł czas, aby użyć wszystkich funkcji po kolei. Zaczniemy od zapytania użytkownika o nazwę zespołu. Potem użyjemy funkcji `get_band`, aby pobrać nazwę, i przede wszystkim identyfikator artysty.

Jeśli operacja się powiedzie, w `band` będziemy mieli słownik z kluczami `"name"` i `"id"` - jeżeli nie, w `band` będzie `None` i jedyne co nam pozostanie do zrobienia to poinformowanie użytkownika, że jego ulubiony zespół nie został znaleziony.

Po pomyślnym pobraniu ID artysty pobierzemy identyfikatory jego wszystkich płyt, a potem w zmiennej `all_tracks` zgromadzimy tytuły wszystkih piosenek, z każdego albumu po kolei.

Gdy tytuły piosenek będą już pobrane, odpytamy odpowiednie API funkcją `get_lyrics` o tekst. Jeśli go nie będzie, wrócimy na początek pętli po kolejną piosenkę. Użycie `not` i `continue` powoduje, że kod nie oddala się za bardzo od lewej krawędzi.

Następnie pobierzemy miejsca, o których wspomina tekst piosenki - funkcja `get_places` zawsze zwraca listę stringów. Nie ma potrzeby sprawdzać czy lista jest pusta - jeśli jest, to pętla `for` wykona się po prostu zero razy.

W środku ww. pętli zapytamy funkcją `get_weather` o pogodę w danym miejscu. Funkcja `get_weather` zwraca albo opis pogody (typu `str`), albo `None` - warto zatem sprawdzić, czy wynik funkcji jest "truthy" - aby pominąć miejsca, dla których pogody nie znaleziono.

## Uruchamianie

Pamiętaj, aby uruchomić wszystkie komórki Jupytera od najwyższej do najniższej - inaczej w pewnym momencie Python stwierdzi, że jakaś funkcja lub zmienna nie istnieje.

Po uruchomieniu poniższej komórki skrypt zapyta Cię o nazwę zespołu, a potem zacznie pracę.

Fragment wyników dla **Train**:
```
Brak tekstu dla Train - For Me, It's You
Pogoda dla SAN FRANCISCO,US: few clouds (Save Me, San Francisco - Train)
Pogoda dla OREGON,US: overcast clouds (Save Me, San Francisco - Train)
Pogoda dla SEATTLE,US: mist (Save Me, San Francisco - Train)
Pogoda dla GOLDEN,US: broken clouds (Save Me, San Francisco - Train)
Pogoda dla MARIN,US: clear sky (Save Me, San Francisco - Train)
Nie znaleziono pogody dla: ALCATRAZ,US
Pogoda dla HIGHWAY,US: clear sky (Save Me, San Francisco - Train)
Pogoda dla FILLMORE,US: clear sky (Save Me, San Francisco - Train)
```


In [ ]:
name = input('Podaj nazwę zespołu')

band = get_band(name)
if band:
    print('Znaleziono:', band['name'])
    
    album_ids = get_album_ids(band['id'])
    
    all_tracks = []
    for aid in album_ids:
        album_tracks = get_tracks(aid)
        all_tracks.extend(album_tracks)
    
    for track_name in all_tracks:
        lyrics = get_lyrics(band['name'], track_name)
        if not lyrics:
            continue

        places = get_places(lyrics)
        
        for place in places:
            weather = get_weather(place)
            if weather:
                print(f'Pogoda dla {place}: {weather} ({track_name} - {band["name"]})')
                
else:
    print('Nie znaleziono:', name)